In [ ]:
#carga de datasets
from utils.DatasetStorage import Dataset
from utils.paths import *

#adaptacion
from utils.adaptacion import create_SDA

# clasificadores
from sklearn.svm import SVC
from utils.clasificacion import *

import os
import time
import pandas as pd
import numpy as np

#variables para guardar los resultados
tipo = pruebas[4]

# Dataset de Twitter

In [ ]:
print pruebas
print datasets[1]
print data_path

In [ ]:
# cargando dataset de Twitter
dataset_path = os.path.join(data_path, datasets[1]+'.pkl')
dataset_object = Dataset().load(dataset_path)

dataset_object.split_dataset(test_size=0.2)

labeled = dataset_object.labeled
domains = dataset_object.domains

In [ ]:
# se obtienen todos los valores X disponibles para realizar adaptacion
X = dataset_object.get_all_X()
X = np.asarray(X)

print "Todos los datos disponibles obtenidos"

In [ ]:
dims = 2000

In [ ]:
#autoencoder, encoder = create_SDA(784, [128, 64, 32, 16, 8])
print "Creando Stacked Autoencoder..."
autoencoder, encoder = create_SDA(dims, [1000])
print "Stacked Autoencoder creado."

In [ ]:
X_train = X[:, :dims]

#TODO: resolver lo de validation data
autoencoder.fit(X_train, X_train,
               epochs=50,
               batch_size=256,
               shuffle=True,
               validation_data=(X_train, X_train))

In [ ]:
#diccionario para mantener los dominios adaptados
adapted = {}

for domain in domains:
    print "Adaptando %s" % domain
    X_tr = np.asarray(labeled[domain]['X_tr'][:, :dims].todense())
    X_ts = np.asarray(labeled[domain]['X_ts'][:, :dims].todense())
    
    tr_reps = encoder.predict(X_tr[:, :dims])
    ts_reps = encoder.predict(X_ts[:, :dims])
    
    adapted[domain] = {
        'X_tr': tr_reps,
        'X_ts': ts_reps
    }

In [ ]:
df = pd.DataFrame(columns=dataframe_columns)

i = 0
# por cada par posible para adaptar
for src in domains:
    for tgt in domains:
        if src is not tgt:
            print "Tarea %d de 2" % (i+1)
            
            #baseline in-domain error
            #e_b(T,T)
            #entrenado en dominio tgt y probado en dominio tgt
            X_tr = np.asarray(labeled[tgt]['X_tr'][:, :dims].todense())
            y_tr = np.asarray(labeled[tgt]['y_tr'].todense()).argmax(axis=1)
            
            X_ts = np.asarray(labeled[tgt]['X_ts'][:, :dims].todense())
            y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)
            
            # se crean las rutas para cargar o crear los modelos
            #model_name = "%s_%s_%s.pkl" % (tipo,src, tgt)
            model_name = "indomain_%s.pkl" % (tgt)
            model_path = os.path.join(models_path, datasets[1], model_name)
            
            #Se realiza una clasificacion, estimando los parametros mediante cv
            svc = load_best_score(model_path, X_tr, y_tr)
            b_error = 1-svc.score(X_ts, y_ts)
            
            #transfer error
            #entrenado en dominio src y probado en dominio tgt adaptados
            X_tr_a = adapted[src]['X_tr']
            y_tr = np.asarray(labeled[src]['y_tr'].todense()).argmax(axis=1)
            
            X_ts_a = adapted[tgt]['X_ts']
            y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)
            
            svc_a = SVC(kernel='linear')
            svc_a = svc_a.fit(X_tr_a, y_tr)
            
            t_error = 1-svc_a.score(X_ts_a, y_ts)
            
            # transfer loss t
            # t_error - b_error
            t_loss = t_error - b_error
            
            tarea = src[0]+'->'+tgt[0]
            df.loc[i] = ['SDA',tarea,src,tgt,b_error*100,t_error*100, t_loss*100]
            i+=1

print "Pruebas completadas."

In [ ]:
df

In [ ]:
new_scores_path = os.path.join(scores_path, tipo, datasets[1]+".csv")

print "Guardando en %s" % new_scores_path
df.to_csv(new_scores_path, columns=df.columns)
print "Resultados guardados."
